In [110]:
import os
import re
import json
import azureml
import requests
import argparse
from pathlib import Path
from azureml.core.run import Run
from azureml.core import Workspace
from azureml.core.model import Model
from azureml.core.image import ContainerImage, Image
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.webservice import Webservice, AciWebservice

print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.0.53


In [101]:
ws = Workspace.from_config()
model = ws.models['seer']

In [102]:
seer_env = CondaDependencies()
seer_env.add_pip_package('tensorflow==2.0.0-beta1')
seer_env.add_pip_package('numpy')
seer_env.add_pip_package('pillow')
seer_env.add_pip_package('requests')

In [113]:
scoring_yml = 'seer_scoring.yml'
with open(scoring_yml,'w') as f:
    print('Writing out {}'.format(scoring_yml))
    f.write(seer_env.serialize_to_string())
    print('Done!')

Writing out seer_scoring.yml
Done!


In [114]:
aciconfig = AciWebservice.deploy_configuration(cpu_cores=1,
                                               memory_gb=1,
                                               description='Transfer learning image classification')

In [115]:
# configure the image
image_config = ContainerImage.image_configuration(execution_script='score.py', 
                                                  runtime='python', 
                                                  conda_file=scoring_yml)


In [116]:
service_name = 'seer-svc'
if service_name in ws.webservices:
    ws.webservices[service_name].delete()
    
service = Webservice.deploy_from_model(workspace=ws,
                                       name=service_name,
                                       deployment_config=aciconfig,
                                       models=[model],
                                       image_config=image_config)

service.wait_for_deployment(show_output=True)

Creating image
Running............................................
Succeeded
Image creation operation finished for image seer-svc:2, operation "Succeeded"
Creating service
Running.....................................................................................................................................................................................................................................................
TimedOut

ERROR - Service deployment polling reached non-successful terminal state, current service state: Unhealthy
Error:
{
  "code": "DeploymentTimedOut",
  "statusCode": 504,
  "message": "The deployment operation polling has TimedOut. The service creation is taking longer than our normal time. We are still trying to achieve the desired state for the web service. Please check the webservice state for the current webservice health. From SDK you can run print(service.state) to know the current state of the webservice."
}

ERROR - Service deployment polling reached non-successful terminal state, current service state: Unhealthy
Error:
{
  "code": "DeploymentTimedOut",
  "statusCode": 504,
  "message": "The deployment operation polling has TimedOut. The service creation is taking longer than our normal time. We are still trying to achieve the desired state for the web service. Please check the webservice state for the current webservice health. From SDK you can run print(service.state) to know t

WebserviceException: Service deployment polling reached non-successful terminal state, current service state: Unhealthy
Error:
{
  "code": "DeploymentTimedOut",
  "statusCode": 504,
  "message": "The deployment operation polling has TimedOut. The service creation is taking longer than our normal time. We are still trying to achieve the desired state for the web service. Please check the webservice state for the current webservice health. From SDK you can run print(service.state) to know the current state of the webservice."
}

In [117]:
with open('deploy.log','w') as f:
    f.write(service.get_logs())
scoring_uri = service.scoring_uri

In [ ]:
tacos = 'https://lh3.googleusercontent.com/-UT5H8nPflkQ/T4tqueyhb_I/AAAAAAAAGl8/1FP7G__Zuys/s640/Lentil+Tacos+close.jpg'
r = requests.post(service, json={'image': tacos})
print(r)
response = r.json()
print(json.dumps(response, indent=4, sort_keys=True))

In [ ]:
burrito = 'https://www.exploreveg.org/files/2015/05/sofritas-burrito.jpeg'
r = requests.post(service, json={'image': tacos})
print(r)
response = r.json()
print(json.dumps(response, indent=4, sort_keys=True))